In [ ]:
class Material():
    def __init__(self, value):
        n, v = value.split(" ")
        self.num = int(n)
        self.val = v
    def __str__(self):
        return "{} {}".format(self.num, self.val)
    def __repr__(self):
        return "{} {}".format(self.num, self.val)
        
assert Material("10 FUEL").num == 10
assert Material("10 FUEL").val == "FUEL"

In [ ]:
class Materials(dict):
    from copy import deepcopy
    def __init__(self, values):
        for output, inputs in values:
            self[output.val] = dict(output=output, inputs=inputs)
        self["ORE"] = dict(output=Material("1 ORE"), inputs=[])
    
    def find_chain(self, m):
        material = self.get(m)
        if material == None:
            return None
        
        material = deepcopy(material)
        
        for i in material.inputs:
            input_material = self.find_chain(i.val)

In [ ]:
class Store:
    
    def __init__(self, materials):
        from collections import Counter
        self.materials = materials
        self.available = Counter()
        self.made = Counter()
    
    def get(self, material, amount):
#         print("Getting {} of {}".format(amount, material))
        element = material["output"].val
        self.available.subtract({element: amount})
        while self.available[element] < 0:
            self.make(**material, amount=-self.available[element])
            
    def make(self, output, inputs, amount):
#         print("Making {} of {} - target {}".format(output.num, output.val, amount))
        import math
        
        target = math.ceil(amount/output.num)
        
        for inpt in inputs:
            material = self.materials[inpt.val]
            self.get(material, target*inpt.num)
        
        self.available.update({output.val: target*output.num})
        self.made.update({output.val: target*output.num})

In [ ]:
def parse_input(value):
    inputs, output = value.split("=>")
    inputs = inputs.split(",")
    inputs = [Material(i.strip()) for i in inputs]
    output = Material(output.strip())
    
    return output, inputs

output, inputs = parse_input("9 ORE => 2 A")
assert output.num == 2
assert output.val == "A"
assert len(inputs) == 1
assert inputs[0].num == 9
assert inputs[0].val == "ORE"

output, inputs = parse_input("9 ORE, 5 GOLD => 2 A")
assert len(inputs) == 2
assert inputs[0].num == 9
assert inputs[0].val == "ORE"
assert inputs[1].num == 5
assert inputs[1].val == "GOLD"


In [ ]:
def parse_sample(value):
    value = value.splitlines()
    value = [parse_input(l) for l in value if len(l)>0]
    return Materials(value)

def make_fuel(materials, amount=1):
    fuel = materials['FUEL']
    store = Store(materials)
    store.get(fuel, amount)
    return store.made

In [ ]:
sample1 = parse_sample("""
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
""")

# 31 Ore
make_fuel(sample1)

In [ ]:
sample2 = parse_sample("""
9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL
""")

# 165 Ore
make_fuel(sample2)

In [ ]:
sample3 = parse_sample("""
157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT
""")

# 13312 Ore
make_fuel(sample3)

In [ ]:
sample4 = parse_sample("""
2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF
""")

# 180697 Ore
make_fuel(sample4)

In [ ]:
sample5 = parse_sample("""
171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX
""")

# 2210736 Ore
make_fuel(sample5)

In [ ]:
with open("14-input.txt", "rt") as FILE:
    data = FILE.read()
materials = parse_sample(data)
make_fuel(materials)

# Part 2

In [ ]:
def find_cycle(materials, total_ore = 1000000000000):
    from tqdm import tqdm, trange
    import numpy as np
    import math

    residuals = 1
    
    production_cyles = []

    store = Store(materials)
    fuel = materials['FUEL']
    iteration = 1
    while residuals > 0:
        store.get(fuel, 1)
        available_list = list(store.available.values())
        production_cyles.append(dict(store.made))
        residuals = np.sum(available_list)
        if iteration % 1000 == 0:
            print(iteration, residuals, available_list)
        iteration += 1

    print(iteration-1, residuals, available_list)
    
    ore_per_cycle = production_cyles[-1]["ORE"]
    fuel_per_cycle = production_cyles[-1]["FUEL"]
    
    cycles = math.floor(total_ore / ore_per_cycle)
    
    fuel_in_cycles = fuel_per_cycle * cycles
    ore_in_cycles = ore_per_cycle * cycles
    
    print(cycles, fuel_in_cycles, ore_in_cycles)
    
    ore_remaining = total_ore - ore_in_cycles
    
    for ix,store in enumerate(production_cyles):
        if ore_remaining - store['ORE'] <= 0:
            print(fuel_in_cycles+production_cyles[ix-1]["FUEL"])
            break
    
#     return ore_per_cycle, total_ore-(cycles*ore_per_cycle)

find_cycle(sample1)  
find_cycle(sample2)

In [ ]:
find_cycle(sample3)

In [ ]:
find_cycle(sample4)

In [ ]:
find_cycle(sample5)

# Part 2 - Attempt 2

It doesn't always converge in a reasonable time - let's use a search algorithm instead

In [ ]:
def fuel_search(materials, total_ore = 1000000000000):
    from scipy.optimize import minimize_scalar
    import math
    
    def search_function(x):
        x = math.ceil(x)
        diff = total_ore - make_fuel(materials, x)["ORE"]
        # If we run out of ore, produce a huge difference
        if diff < 0:
            diff = 1e100

        return diff
    
    x = minimize_scalar(search_function, method='brent')
    print(x)
    print("Fuel made", math.ceil(x.x))
    
fuel_search(sample3)

In [ ]:
fuel_search(sample4)

In [ ]:
fuel_search(sample5)

In [ ]:
fuel_search(materials)